# Results Plot

This notebook takes in the trained posteriors and generates various plots used in the paper, namely for the results section. 
We produce the posterior plot (with predictives) for one layer.
Additionally we produce the posterior comparison plot between different layers (and validation data, when available).

We also evaluated the expected MSE of the posterior predictive relative to the ground truths in this notebook.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from omegaconf import DictConfig,OmegaConf
from pathlib import Path
import pickle
import torch
import numpy as np
from sbi_ice.loaders import ShortProfileLoader
import sbi.analysis as analysis
from sbi_ice.utils import posterior_utils,modelling_utils,plotting_utils,misc
import matplotlib.pyplot as plt
from tueplots import figsizes

data_dir,output_dir = misc.get_data_output_dirs()
work_folder = misc.get_project_root()
color_opts = plotting_utils.setup_plots()



## Load Posterior Config for posterior we want to work with


In [ ]:
#seeds = [layer_0_seed_101,layer_1_seed_100,layer_2_seed_101,layer_3_seed_103] #Ekstrom exp 2 all_final seeds
#seeds = [layer_0_seed_1100,layer_1_seed_1102,layer_2_seed_1104,layer_3_seed_1101] #Synthetic_long exp3 all_final seeds
#seeds = [layer_0_seed_1200,layer_1_seed_1203] #Synthetic exp3 all_final_v3 seeds
# shelf = "Synthetic_long"
# exp = "exp3"
# name = "all_final"
# seed = "layer_3_seed_1101"
# name = "all_final_v3"
# seed = "layer_1_seed_1203"
shelf = "Ekstrom"
exp = "exp2"
name = "all_final"
seed = "layer_0_seed_101"
fol = Path(output_dir , shelf, exp, "sbi_sims/post_predictives" , name,seed)
cfg_path = Path(fol,"config.yaml")
cfg = OmegaConf.load(cfg_path)
config_fol = cfg.posterior_config_fol
config_fol = Path(output_dir,shelf,exp,"sbi_sims/posteriors",name)
print("Path to config file: " , Path(config_fol,str(cfg.name),"config.yaml"))


n_post_samples = cfg.n_post_samples
n_predictive_sims = cfg.n_predictive_sims
print("number of predictive sims loaded: " , n_predictive_sims)
overwrite = cfg.overwrite_saved_sims
posterior_config = OmegaConf.load(Path(config_fol,str(cfg.name),"config.yaml"))
print("Path to posterior config file: " ,posterior_config)


paths = posterior_config.paths
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
output_fol = os.getcwd()
print("Path to output folder where sims are stored: ", output_fol)

shelf_setup_path = paths.shelf_setup_path
print("Setup file for shelf: ", shelf_setup_path)


shelf_folder_path = paths.shelf_folder_path
exp_path = paths.exp_path

print("Read Config files")
loader = ShortProfileLoader.ShortProfileLoader(Path(work_folder,shelf_setup_path),Path(work_folder,shelf_folder_path),exp_path,sims_path="layer_sims",gt_version = posterior_config.gt_version)
loader._jobs = [i for i in range(1,200)]
loader._num_sims = [1000 for i in range(1,200)]
loader.total_sims = 1000*200
print("Set up data loader")


## Load posterior network, true values, and posterior predictive simulations

In [ ]:
with open(Path(config_fol,str(cfg.name),"inference.p"), "rb") as f:
    if device.type == "cpu":
        out = posterior_utils.CPU_Unpickler(f).load()
    else:
        out = pickle.load(f)


inference = out["inference"]
prior = out["prior"]
layer_mask = out["layer_mask"]
smb_mask = out["smb_mask"]
true_layer = torch.tensor(loader.real_layers[cfg.layer_idx][layer_mask]).float()
posterior = inference.build_posterior(inference._neural_net.to("cpu"))
posterior.set_default_x(true_layer)
samples = posterior.sample((n_post_samples,))
try:
    true_smb = loader._true_smb_const_mean + loader._true_smb_var*loader._true_smb_unperturbed
    #true_smb = true_smb[smb_mask]
    true_smb = modelling_utils.regrid(loader._true_xmb,true_smb,loader.x)
except:
    true_smb=None
spatial_samples = samples
print(spatial_samples.shape)

print("Reading posterior predictive sims from file...")
with open(Path(fol,"post_predictive.p"), "rb") as f:
    out = pickle.load(f)
    bmb_samples = out["bmb_samples"]
    best_layers = out["best_layers"]
    norms = out["norms"]
    ages = out["ages"]
    active_trackers = out["active_trackers"]

print("Finished posterior predictive sims")
print(best_layers.shape)



In [ ]:
print(torch.mean(true_layer))
print(np.mean(loader.surface-loader.real_layers[cfg.layer_idx]))


### Load prior simulations for comparison plots

In [ ]:
contour_arrays,norm_arrays,age_arrays,smb_unperturbed_all,smb_cnst_means_all,smb_sds_all,smb_all,bmb_all = \
    loader.load_training_data(layers_fname = posterior_config.layers_fname,mb_fname = posterior_config.mb_fname)


# Plot Main Results plot for this shelf/experiment/IRH

In [ ]:
true_ages = [49.5,  99.5,  149.5, 299.5] #Synthetic_long exp3
# true_age = true_ages[cfg.layer_idx]
true_age = None
layer_idx = cfg.layer_idx

plot_layer_mask = layer_mask
#plot_layer_mask = np.ones_like(layer_mask).astype(bool)
perm = torch.randperm(smb_all.size(0))
perm_idx = perm[:1000]
prior_samples = smb_all[perm_idx][:,smb_mask]
prior_spatial_samples = prior_samples
tmb = smb_all[0].numpy() + bmb_all[0].numpy()
title = shelf + " Layer {} Posterior Predictive".format(layer_idx+1)
print(contour_arrays[cfg.layer_idx,:n_predictive_sims,:].shape)
fig,axs = plotting_utils.plot_posterior_nice(x = loader.x,
                                            mb_mask = smb_mask,
                                            tmb = tmb,
                                            prior_smb_samples = prior_spatial_samples,
                                            posterior_smb_samples = spatial_samples,
                                            layer_mask = plot_layer_mask,
                                            LMI_boundary = loader.x[layer_mask][0],
                                            prior_layer_samples= contour_arrays[cfg.layer_idx,perm_idx,:],
                                            prior_layer_ages=age_arrays[cfg.layer_idx,perm_idx],
                                            posterior_layer_samples=best_layers,
                                            posterior_layer_ages=ages,
                                            true_layer=loader._real_layers[cfg.layer_idx],
                                            shelf_base=loader.base.flatten(),
                                            shelf_surface=loader.surface.flatten(),
                                            true_smb=true_smb,
                                            true_age=true_age,
                                            plot_samples=False,
                                            title=title,
                                            )
fig_name = Path(output_dir,"paper_figures",shelf,name + "_" + seed + "_" + "predictive.pdf")
fig_name.parent.mkdir(parents=True,exist_ok=True)
fig.savefig(fig_name)



In [ ]:
print(np.percentile(ages,16))
print(np.percentile(ages,50))
print(np.percentile(ages,84))
print(ages)

In [ ]:
bmb_samples = spatial_samples - torch.Tensor(tmb[smb_mask]).unsqueeze(0)
print(bmb_samples.mean(0))
print(bmb_samples.mean())

# Plot Pairplot of SMB

In [ ]:
limits = [[-1.0,2.0] for i in range(0,spatial_samples.shape[1])]
subset = [5*i for i in range(10)]
labels = ["{:.2f} km".format(1.25*np.round(loader.x[10*i]/1250,0)) for i in range(0,spatial_samples.shape[1])]

# fig,axs = analysis.pairplot([spatial_samples,prior_spatial_samples],subset=subset,limits=limits,labels=labels,
#                             samples_colors=[color_opts["colors"]["posterior"],color_opts["colors"]["prior"]],
#                             upper="contour",contour_offdiag={"levels": [0.68,0.95], "percentile": True},
#                             points_offdiag={"marker": ".","markersize": 5,},points_colors=color_opts["colors"]["observation"])

# fig_name = Path(output_dir,"paper_figures",shelf,name + "_" + seed + "_" + "pairplot.pdf")
# fig_name.parent.mkdir(parents=True,exist_ok=True)
# fig.savefig(fig_name)


# Plot Yearly Samples of Kottas Traverse Accumulation Data alongside SMB posterior

In [ ]:
kottas_fname =  Path(data_dir,"Ekstrom","kottas_mbs.p")
with open(kottas_fname, "rb") as f:
    kottas_smb = pickle.load(f)
plt.rcParams.update(figsizes.icml2022_full(height_to_width_ratio=1/1.1))
fig, ax = plt.subplots()
color_opts = plotting_utils.color_opts

ax.hlines(0,loader.x[smb_mask][0]/1e3,loader.x[smb_mask][-1]/1e3,color="black",linestyle="--")
kt1, = ax.plot(kottas_smb["kottas_xmb"]/1e3,kottas_smb["kottas_time_mean_smb"],color=color_opts["colors"]["observation"])
for i,signal in enumerate(kottas_smb["all_years_smb"]):
    if i ==0:
        label = "Kottas years"
    else:
        label = None
    kts, = ax.plot(kottas_smb["kottas_xmb"]/1e3,signal,color=color_opts["colors"]["observation"],alpha=0.25,label=None)

percentiles = [5,95]
post_mean_smb = torch.mean(spatial_samples,axis=0)
post_uq_smb = torch.quantile(spatial_samples,percentiles[1]/100,axis=0)
post_lq_smb = torch.quantile(spatial_samples,percentiles[0]/100,axis=0)

po1, = ax.plot(loader.x[smb_mask]/1e3,post_mean_smb,color=color_opts["colors"]["posterior"])
po2 = ax.fill_between(loader.x[smb_mask]/1e3,post_lq_smb,post_uq_smb,color=color_opts["colors"]["posterior"],alpha=0.2,linewidth=0.0)
ax.vlines(loader.x[layer_mask][0]/1e3,ymin=0.05,ymax=0.95,linestyles="dashed",
          color=color_opts["colors"]["boundary_condition"],
          transform=ax.get_xaxis_transform())

handles = [(kt1,),(kts,),(po1,po2)]
labels = ["Kottas Mean","Kottas Yearly Data","Posterior"]

ax.spines['bottom'].set_bounds(loader.x[smb_mask][0]/1e3-0.001,loader.x[smb_mask][-1]/1e3+2)
ax.set_xlabel("Distance from GL [km]")
ax.set_ylabel("Surface accumulation [m/a]")
ax.legend(handles=handles,labels=labels,loc = "best")

fig_name = Path(output_dir,"paper_figures",shelf,"Kottas_data.pdf")
fig_name.parent.mkdir(parents=True,exist_ok=True)
fig.savefig(fig_name)


# Plot comparison of posteriors for all 4 IRHs

In [ ]:



posteriors = []
observations = []
layers = []

prior_mismatches = []
post_mismatches = []
#seeds = ["layer_0_seed_203","layer_1_seed_201","layer_2_seed_203","layer_3_seed_204"]
#seeds = ["layer_0_seed_2000","layer_1_seed_2000","layer_2_seed_2000","layer_3_seed_2000"]
#seeds = ["layer_0_seed_2100","layer_1_seed_2100","layer_2_seed_2100","layer_3_seed_2100"]
# seeds = ["layer_0_seed_400","layer_1_seed_400","layer_2_seed_400","layer_3_seed_400"]
seeds = ["layer_0_seed_101","layer_1_seed_100","layer_2_seed_101","layer_3_seed_103"]
# seeds = ["layer_0_seed_1100","layer_1_seed_1102","layer_2_seed_1104","layer_3_seed_1101"]
for idx,seed in enumerate(seeds):
    true_ages = [49.5,  99.5,  149.5, 299.5] #Synthetic_long exp3 gt_v3
    fol = Path(output_dir , shelf, exp, "sbi_sims/post_predictives" , name,seed)
    cfg_path = Path(fol,"config.yaml")
    cfg = OmegaConf.load(cfg_path)
    config_fol = cfg.posterior_config_fol
    config_fol = Path(output_dir,shelf,exp,"sbi_sims/posteriors",name)
    n_post_samples = cfg.n_post_samples
    n_predictive_sims = cfg.n_predictive_sims
    overwrite = cfg.overwrite_saved_sims
    posterior_config = OmegaConf.load(Path(config_fol,str(cfg.name),"config.yaml"))
    paths = posterior_config.paths
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    output_fol = os.getcwd()
    with open(Path(config_fol,str(cfg.name),"inference.p"), "rb") as f:
        if device.type == "cpu":
            out = posterior_utils.CPU_Unpickler(f).load()
        else:
            out = pickle.load(f)
    inference = out["inference"]
    layer_mask = out["layer_mask"]
    true_layer = torch.tensor(loader.real_layers[cfg.layer_idx]).float()
    posterior = inference.build_posterior(inference._neural_net.to("cpu"))
    posterior.set_default_x(true_layer[layer_mask])
    posteriors.append(posterior)
    observations.append(true_layer)
    with open(Path(fol,"post_predictive.p"), "rb") as f:
        out = pickle.load(f)
        layer_ages = out["ages"]


    nan_mask = ~torch.isnan(true_layer)
    layer_entry = {"layer_x":loader.x[nan_mask]/1e3,"layer_depth":loader.surface[nan_mask] - true_layer[nan_mask].numpy(),
                   "layer_age":np.median(layer_ages) if shelf == "Ekstrom" else true_ages[idx],"total_thickness":loader.surface[nan_mask] - loader.base[nan_mask],
                   "LMI_x":loader.x[layer_mask]/1e3}
    layers.append(layer_entry)


    with open(Path(fol,"post_predictive.p"), "rb") as f:
        out = pickle.load(f)
        bmb_samples = out["bmb_samples"]
        best_layers = out["best_layers"]
        norms = out["norms"]
        ages = out["ages"]
        active_trackers = out["active_trackers"]

    contour_arrays,norm_arrays,age_arrays,smb_unperturbed_all,smb_cnst_means_all,smb_sds_all,smb_all,bmb_all = loader.load_training_data(layers_fname = posterior_config.layers_fname,mb_fname = posterior_config.mb_fname)
    perm = torch.randperm(smb_all.size(0))
    perm_idx = perm[:1000]
    prior_layers = contour_arrays[idx,perm_idx,:]

    prior_mismatch = prior_layers[:,layer_mask].numpy() - true_layer[layer_mask].numpy()
    post_mismatch = best_layers - true_layer[layer_mask].numpy()
    prior_mismatches.append(prior_mismatch)
    post_mismatches.append(post_mismatch)


if shelf == "Ekstrom":
    fig,axs = plotting_utils.compare_posteriors(loader.x[smb_mask]/1e3,posteriors,layers,kottas_smb=kottas_smb)
elif shelf == "Synthetic_long":
    fig,axs = plotting_utils.compare_posteriors(loader.x[smb_mask]/1e3,posteriors,layers,real_smb = true_smb)
# fig_name = Path(output_dir,"paper_figures",shelf,"post_comparison.pdf")
# fig_name.parent.mkdir(parents=True,exist_ok=True)
# fig.savefig(fig_name)

## Calculate RMSE between posteriors and real layer data

In [ ]:
percentiles = [5,95]
table = [["IRH number",1,2,3,4]]
prior_RMSE_mean = ["prior RMSE average"]
prior_RMSE_uq = ["prior RMSE {:.0f}% UQ".format(percentiles[1])]
prior_RMSE_lq = ["prior RMSE {:.0f}% LQ".format(percentiles[0])]
prior_RMSE_sd = ["prior RMSE SD"]
posterior_RMSE_mean = ["posterior RMSE average"]
posterior_RMSE_uq = ["posterior RMSE {:.0f}% UQ".format(percentiles[1])]
posterior_RMSE_lq = ["posterior RMSE {:.0f}% LQ".format(percentiles[0])]
posterior_RMSE_sd = ["posterior RMSE SD"]

for i in range(0,len(prior_mismatches)):
    print(prior_mismatches[i].shape)
    prior_RMSE_mean.append(np.mean(np.sqrt(np.mean(np.square(prior_mismatches[i]),axis=1))))
    prior_RMSE_uq.append(np.quantile(np.sqrt(np.mean(np.square(prior_mismatches[i]),axis=1)),percentiles[1]/100))
    prior_RMSE_lq.append(np.quantile(np.sqrt(np.mean(np.square(prior_mismatches[i]),axis=1)),percentiles[0]/100))
    prior_RMSE_sd.append(np.std(np.sqrt(np.mean(np.square(prior_mismatches[i]),axis=1))))
    posterior_RMSE_mean.append(np.mean(np.sqrt(np.mean(np.square(post_mismatches[i]),axis=1))))
    posterior_RMSE_uq.append(np.quantile(np.sqrt(np.mean(np.square(post_mismatches[i]),axis=1)),percentiles[1]/100))
    posterior_RMSE_lq.append(np.quantile(np.sqrt(np.mean(np.square(post_mismatches[i]),axis=1)),percentiles[0]/100))
    posterior_RMSE_sd.append(np.std(np.sqrt(np.mean(np.square(post_mismatches[i]),axis=1))))

table.append(prior_RMSE_mean)
#table.append(prior_RMSE_uq)
#table.append(prior_RMSE_lq)
table.append(prior_RMSE_sd)
table.append(posterior_RMSE_mean)
#table.append(posterior_RMSE_uq)
#table.append(posterior_RMSE_lq)
table.append(posterior_RMSE_sd)

table = list(map(list,zip(*table)))



In [ ]:
!pip install tabulate

In [ ]:
from tabulate import tabulate 
print(tabulate(table,headers="firstrow"))
#output to latex
print(tabulate(table,headers="firstrow",tablefmt="latex_raw",floatfmt=".3f"))